## Introduction to Human In The Loop in Agent Framework

Workflows in the Microsoft Agent Framework allow developers to design, monitor, and control how multiple AI agents and logic components interact to complete complex tasks. They bring structure, flexibility, and transparency to agent-driven applications.

One of the ways to bring in control and manage agent workflows is to add human approvals. When agents require any user input, for example to approve a function call, this is referred to as a **human-in-the-loop** pattern. 

An agent run that requires user input will complete with a response that indicates what input is required from the user instead of completing with a final answer. The caller of the agent is then responsible for getting the required input from the user and passing it back to the agent as part of a new agent run.


### Example 1 - Function Tools with Human-In-The-Loop approvals



In [1]:
import os
from dotenv import load_dotenv
from agent_framework.azure import AzureOpenAIChatClient

load_dotenv()
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")
deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

chat_client=AzureOpenAIChatClient(
        endpoint=endpoint,
        api_key=api_key,
        api_version=api_version,
        deployment_name=deployment,
    )

In [ ]:
from typing import Annotated
from agent_framework import ai_function, ChatMessage, Role, ChatAgent 
from agent_framework.azure import AzureOpenAIResponsesClient


@ai_function
def get_weather(location: Annotated[str, "The city and state, e.g. San Francisco, CA"]) -> str:
    """Get the current weather for a given location."""
    return f"The weather in {location} is sunny with a high of 30°C."

@ai_function(approval_mode="always_require")
def get_weather_detail(location: Annotated[str, "The city and state, e.g. San Francisco, CA"]) -> str:
    """Get detailed weather information for a given location."""
    return f"The weather in {location} is cloudy with a high of 15°C, humidity 88%."


agent1 = ChatAgent(
    chat_client=chat_client,
    name="WeatherAgent",
    instructions="You are a helpful weather assistant.",
    tools=[get_weather_detail, get_weather],
)

result = await agent1.run("What is the detailed weather like in Amsterdam?")

if result.user_input_requests:
    for user_input_needed in result.user_input_requests:
        print(f"Function: {user_input_needed.function_call.name}")
        print(f"Arguments: {user_input_needed.function_call.arguments}")

        
    # Get user approval (in a real application, this would be interactive)
    user_approval = False  # or False to reject

    # Create the approval response
    approval_message = ChatMessage(
        role=Role.USER, 
        contents=[user_input_needed.create_response(user_approval)]
    )

    final_result = await agent1.run([
        "What is the detailed weather like in Amsterdam?",
        ChatMessage(role=Role.ASSISTANT, contents=[user_input_needed]),
        approval_message
    ])

print(final_result.text)

Function: get_weather_detail
Arguments: {"location":"Amsterdam"}
The current weather in Amsterdam is sunny with a high temperature of 30°C. If you would like more detailed weather information, please let me know!


In [ ]:
from dataclasses import dataclass
# Custom task - submit expenses - if we have a small expense of < 100 eur, we can just add it to the expense list.
# If it's a bigger one, before submission you need to provide a description and send it off to your manager for approval
# tool to approve the expense

# list of mock entries 

#input for the looped approvals:
@dataclass
class Expense:
    amount: float
    type: str
    description: str
    status: str


@ai_function
def list_pending_expenses()        

@ai_function(approval_mode="always_require")
def add_expense(amount: Annotated[int, "The expense amount"]) -> str:
    """Add the expense to the database"""
    if amount > 150:



# WORKFLOW:
# in main - some expenses already, then we start an agent and say "add expense" or "show me unprocessed ones"
# processing expenses means approving them rightaway or rejecting them
# add_expense, reject_expense, approve, list_pending, list_rejected, 



### Request and Response

#### Recap - Message-Centric Execution Model

In AF, agent execution is fundamentally about the **data flow** - a workflow will keep running as long as there is data being passed. Agents don't "call" each other or pass control in a traditional procedural sense. Instead, they communicate through typed messages flowing between executors through edges.

**Executors** are where computation happens. When an executor emits a message, it flows along the edges to downstream executors which activate when their dependent data becomes available.

Agents communicate with applications via different message content types (`TextContent`, `FunctionCallContent`, `FunctionApprovalRequestContent`, etc.). Messages are represented by the `ChatMessage` class and all content type classes inherit from the base `BaseContent` class.

For a detailed list of available content types and their usage, refer to the folowing (documentation)[https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/running-agents?pivots=programming-language-python].


#### User Approval Content

The content of the message can also be used to let the application approve or disapprove some actions the agent wants to take.
**User Approval Content** type enables human-in-the-loop approvals within the standard message flow.


The approval mechanism intercepts the normal tool invocation flow, pausing execution until human approval is granted or denied:

<img src="images/reqresp.png" alt="User Approval Content Type" style="width:70%; height:40%; margin:20px auto">

When a ChatAgent needs to invoke a Tool that requires approval:

1. **Initial Inference**: The ChatAgent uses the Model to make an inference and determines it needs to invoke the Tool.

2. **Approval Request**: Before executing, the Tool sends a request back to the ChatAgent, which surfaces to the Application as `FunctionApprovalRequestContent` (a special ChatMessage content type).

3. **User Response**: The Application presents this approval request to the user, who responds (Yes/No). This response is sent back to the ChatAgent as `FunctionApprovalResponseContent`.

4. **Tool Execution**: Once the ChatAgent receives approval, the Tool executes and returns its result to the ChatAgent.

5. **Final Inference**: The Model makes another inference using the Tool's result to generate a natural language response as `TextContent`.

6. **Complete Response**: The Application receives both the `FunctionResultContent` (the raw tool output) and the `TextContent` (the Model's interpretation of that result).

#### User Approval Importance

User approvals are necessary in many cases, especially when tool execution involves external resources or has side effects. Common scenarios include:

- **Irreversible operations**: Data deletion, financial transactions, resource provisioning
- **External system access**: Database modifications, API calls, file system operations
- **Security-sensitive actions**: Credential access, permission changes, privileged operations

When a tool requires approval, the agent pauses execution and transfers control to the application. The agent preserves its state (conversation history, pending actions, context) and waits for the application's decision before proceeding. This mechanism ensures critical operations require explicit human authorization while maintaining the agent's execution context for immediate resumption.

As covered in Chapter 1, **Agent Middleware** provides a general-purpose mechanism for intercepting agent interactions at three levels: agent (intercepts entire agent execution), function (intercepts individual tool invocations), and chat (intercepts LLM calls). You can think of the process of user approval during tool invocation as *tool middleware*. 

TODO1 - requests and responses:
https://learn.microsoft.com/en-us/agent-framework/tutorials/workflows/requests-and-responses?pivots=programming-language-python


TODO2 - https://github.com/microsoft/agent-framework/blob/main/python/samples/getting_started/workflows/agents/workflow_as_agent_human_in_the_loop.py

TODO3 - https://github.com/microsoft/agent-framework/blob/main/python/samples/getting_started/workflows/human-in-the-loop/guessing_game_with_human_input.py
